In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
import csv

#http://www.tweepy.org/
import tweepy

#Get your Twitter API credentials and enter them here
consumer_key = 
consumer_secret = 
access_key = 
access_secret = 

In [23]:
#method to get a user's last 100 tweets
def get_tweets(username):

	#http://tweepy.readthedocs.org/en/v3.1.0/getting_started.html#api
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)

	#set count to however many tweets you want; twitter only allows 200 at once
	number_of_tweets = 100

	#get tweets
	tweets = api.user_timeline(screen_name = username,count = number_of_tweets)
    
    followers = 

	#create array of tweet information: username, tweet id, date/time, text
	tweets_for_csv = [[username, tweet.id_str,  tweet.favorite_count, tweet.retweet_count, tweet.created_at, tweet.text.encode("utf-8")] for tweet in tweets]

	#write to a new csv file from the array of tweets
	print "writing to {0}_tweets.csv".format(username)
	with open("{0}_tweets.csv".format(username) , 'w+') as file:
		writer = csv.writer(file, delimiter='|')
		writer.writerows(tweets_for_csv)




In [ ]:
####if we're running this as a script
#if __name__ == '__main__':

 #   #get tweets for username passed at command line
 #   if len(sys.argv) == 2:
 #       get_tweets(sys.argv[1])
 #   else:
 #       print "Error: enter one username"

    ####alternative method: loop through multiple users
	#### users = ['user1','user2']

	#### for user in users:
	#### 	get_tweets(user)

In [24]:
get_tweets("HillaryClinton")

writing to HillaryClinton_tweets.csv


In [26]:
df = pd.read_csv('HillaryClinton_tweets.csv',sep ='|',names=['username', 'id_str', 'favorite_count', 'retweet_count', 'created_at', 'text'])
df

,username,id_str,favorite_count,retweet_count,created_at,text
0,HillaryClinton,741694471000072192,1413,813,2016-06-11 18:11:56,Trump University: Pad Donald's pockets and put...
1,HillaryClinton,741684990073667585,2629,997,2016-06-11 17:34:16,.@realDonaldTrump is temperamentally unfit to ...
2,HillaryClinton,741634938718552064,0,873,2016-06-11 14:15:22,RT @sierraclub: The divide between @HillaryCli...
3,HillaryClinton,741478368676192256,3468,1061,2016-06-11 03:53:13,"Lights, camera, history.\r\nhttps://t.co/C7XFF..."
4,HillaryClinton,741443616594395137,8874,5268,2016-06-11 01:35:08,A man with this much contempt and disrespect f...
5,HillaryClinton,741425791825776640,7322,2382,2016-06-11 00:24:18,.@realDonaldTrump says he’ll be “the best for ...
6,HillaryClinton,741409978376957952,12189,5085,2016-06-10 23:21:28,"""I am the least racist person that you’ve ever..."
7,HillaryClinton,741396683360239616,4054,1304,2016-06-10 22:28:38,"""She’s a fighter. She’s out there. She’s tough..."
8,HillaryClinton,741385713439113216,0,942,2016-06-10 21:45:03,"RT @mikereedschmidt: Hey @realDonaldTrump, mee..."
9,HillaryClinton,741378346404044800,5490,3326,2016-06-10 21:15:46,Trump can’t say he treats women with respect.\...


In [30]:
username = "hillaryclinton"

#http://tweepy.readthedocs.org/en/v3.1.0/getting_started.html#api
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#set count to however many tweets you want; twitter only allows 200 at once
number_of_tweets = 100

#get tweets
tweets = api.user_timeline(screen_name = username,count = number_of_tweets)

In [31]:
tweet = tweets[0]

In [32]:
tweet

Status(contributors=None, truncated=False, text=u"Trump University: Pad Donald's pockets and put your own finances at risk\u2014all for the low price of $35,000!\nhttps://t.co/6NHo9RTyHz", is_quote_status=False, in_reply_to_status_id=None, id=741694471000072192L, favorite_count=1651, _api=<tweepy.api.API object at 0x000000000418C9E8>, author=User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=False, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': False, u'default_profile_image': False, u'id': 1339835893, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': True, u'profile_text_color': u'000000', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/740354638411616258/wRU3ncoU_normal.jpg', u'profile_sidebar_fill_color': u'000000', u'entities': {u'url': {u'urls': [{u'url': u'https://t.co/qqt83R8Fqb', u'indices': [0, 23], u'expanded_ur